# TPOT training

I have realized that TPOT takes a really long time to run, days. This notebook contains the TPOT runs I have tried. Another notebook contains the inspection of the results, as I can do it while running using checkpoints.

## Setup

In [1]:
import pickle

In [2]:
import sys
sys.path.append('..')
from utils import *

In [3]:
x_train, y_train, eras_train = train_data(weras=True)
x_val, y_val = val_data()
bl = baseline(x_train, y_train, x_val, y_val)
bl

0.69245942891354983

## TPOT

In [4]:
from tpot import TPOTClassifier

### TPOT light

In [6]:
model = TPOTClassifier(
    config_dict='TPOT light',
    scoring='neg_log_loss',
    periodic_checkpoint_folder='tpot',
    n_jobs=8,
    verbosity=3,
)

19 operators have been imported by TPOT.


In [ ]:
model.fit(x_train, y_train, groups=eras_train)

Optimization Progress:   0%|          | 0/10100 [00:00<?, ?pipeline/s]

In [ ]:
y_val_pred = model.predict_proba(x_val)
validate(y_val, y_val_pred, bl, True)

In [ ]:
model.export('tpot.py')

In [ ]:
with open('tpot.pkl', 'wb') as auto_sklearn_file:
    pickle.dump(model, auto_sklearn_file)

## Auto-sklearn

In [ ]:
from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import log_loss

In [ ]:
model = AutoSklearnClassifier(
    time_left_for_this_task=4*60*60,  # 4h
    ml_memory_limit=6144,             # 6GB
)

In [ ]:
%time model.fit(x_train, y_train, metric=log_loss)

In [ ]:
model.show_models()

In [ ]:
model.get_models_with_weights()

In [ ]:
y_val_pred = model.predict_proba(x_val)
validate(y_val, y_val_pred, bl, True)

In [ ]:
with open('auto_sklearn.pkl', 'wb') as auto_sklearn_file:
    pickle.dump(model, auto_sklearn_file)